# ComScore data: Explore demographics

In [1]:
!ls ../data/comscore/2017

2017transactions.csv  codebook.pdf  demographics.csv


In [2]:
%pip install census us matplotlib pyzipcode joblib

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np
from census import Census
import us
c = Census("14a343e581b5dd1df551f4cd87f31aede50ec908")

%matplotlib inline

In [9]:
def get_zip_state_fips(zipcode, err=False):
    """returns a state FIPS code for a given zip.
    
    if it can't find the zip code, return np.nan if err=False.
    
    parameters
    ----------
    err: boolean. True if function should raise a ValueError if it can't find a zip code. False by default.
    """
    from pyzipcode import ZipCodeDatabase
    zcdb = ZipCodeDatabase()
    if zipcode not in zcdb:
        if err:
            raise ValueError("Zip code not found")
        else:
            return np.nan
    zipcode = zcdb[zipcode]
    state = us.states.lookup(zipcode.state)
    return state.fips

In [209]:
demo_df = (pd.read_csv("../data/comscore/2017/demographics.csv", dtype={'zip_code': str})
           .drop('machine_id', axis=1)
           .dropna(subset=['zip_code'])
           .replace({'hoh_most_education': {99: np.nan}})
           # make sure zips are zip5
           .assign(zip_code=lambda x: x.zip_code.apply(lambda z: '0' + z if len(z) < 5 else z))
           # get state fips code for each zip
           .assign(state_fips=lambda x: x.zip_code.apply(get_state_fips))
           .dropna(subset=['state_fips']) # drops ~600 rows
     )

In [ ]:
demo_df.head()

## Retrieving census data for states

Let's test getting relevant census data. I went [here](https://api.census.gov/data/2017/acs/acs5/profile/variables.html) and looked up relevant variables and just hand-coded them below. I added a mapping to the relevant comscore coding.

TODO: 

- [ ] Add additional codes for other variables
- [ ] Change to state-level census data—we decided not to use zip

In [13]:
census_mapping = pd.DataFrame([
    # education level. 
    ['DP02_0064PE', 4, 'bachelors degree', 'hoh_most_education'],
    ['DP02_0061PE', 1, 'high school graduate', 'hoh_most_education'],
    ['DP02_0065PE', 5, 'graduate degree', 'hoh_most_education'],
    ['DP02_0062PE', 2, 'some college, no degree', 'hoh_most_education'],
    ['DP02_0063PE', 3, 'associates degree', 'hoh_most_education'],
    # two rows for comscore-coded zero. Should sum for comparison.
    ['DP02_0060PE', 0, '9-12 grade no diploma', 'hoh_most_education'],
    ['DP02_0059PE', 0, 'less than 9th grade', 'hoh_most_education'],
    ['DP05_0077PE', 1, 'non-hispanic white', 'racial_background'],
    ['DP05_0078PE', 2, 'non-hispanic black', 'racial_background'],
    ['DP05_0080PE', 3, 'non-hispanic asian', 'racial_background'],
], columns=['census_code', 'comscore_code', 'description', 'comscore_column'])

In [138]:
def get_census_data_for_zip(zipcode, var_names, year=2017):
    """retrieves census variables for a given zipcode and year from the ACS
    
    parameters
    ----------
    zipcode: string. zip code to search for
    var_names: list. list of variable names from the ACS.
    
    returns
    -------
    A dict with a key for each var in var_names where the value
    is the corresponding value for the zipcode, plus a key for `state` with 
    the corresponding FIPS code and a key for `zip code tabulation area` 
    that parrots the zip.
    
    example
    -------
    >>> get_census_data_for_zip('02130', ['DP05_0080PE'])
    """
    state_fips = get_zip_state_fips(zipcode)
    if not state_fips:
        return None
    try:
        result = c.acs5dp.state_zipcode(
            var_names,
            state_fips,
            zipcode,
            year=year)[0]
        return result
    except Exception as e:
        print("Couldn't retrieve census data for zip {}: {}".format(zipcode, e))
        return None

r = demo_df.iloc[0]
zips = list(set(demo_df.zip_code))
zip_demos = c.acs5dp.state(list(census_mapping.census_code.values), r.state_fips, year=2017)

In [16]:
census_vars = list(census_mapping.census_code.values)

It takes a while, so run in parallel (way faster)

In [144]:
from joblib import Parallel, delayed
N_CORES = 24

states = list(set(demo_df.state_fips))

%time state_census_data = Parallel(n_jobs=N_CORES)(delayed(c.acs5dp.state)(census_vars, state, year=2017) for state in states)
census_df = pd.DataFrame([s[0] for s in state_census_data if s is not None])

%mkdir ../output
census_df.to_csv("../output/state_census.csv")

CPU times: user 64.5 ms, sys: 14.2 ms, total: 78.7 ms
Wall time: 3.45 s
mkdir: cannot create directory ‘../output’: File exists


In [147]:
census_df = pd.read_csv("../output/state_census.csv", index_col=0)

In [149]:
census_df.head(2)

,DP02_0064PE,DP02_0061PE,DP02_0065PE,DP02_0062PE,DP02_0063PE,DP02_0060PE,DP02_0059PE,DP05_0077PE,DP05_0078PE,DP05_0080PE,state
0,13.3,30.4,8.0,22.6,9.1,11.0,5.6,57.0,37.5,1.0,28
1,23.4,24.7,18.7,15.8,7.7,5.1,4.6,72.9,6.7,6.2,25


## Create corresponding zip code aggregates from comscore demographics

In [150]:
demo_df.head()

,hoh_most_education,census_region,household_size,hoh_oldest_age,household_income,children,racial_background,connection_speed,country_of_origin,zip_code,state_fips
0,NaN,4,5,8,16,0,1,1,0,98611,53
1,3.0,4,3,8,14,0,1,1,0,80910,08
2,4.0,1,3,11,16,1,1,1,0,07076,34
3,3.0,2,1,9,16,0,1,1,0,62554,17
5,1.0,4,3,7,12,0,1,1,1,93436,06


In [151]:
state_df = (pd.melt(demo_df,
       id_vars=['state_fips'],
       value_vars=['hoh_most_education', 'racial_background'],
       value_name='val',
       var_name='var_name').groupby(['state_fips', 'var_name', 'val'])).val.agg(['count'])
state_df

count
state_fips var_name           val       
01         hoh_most_education 1.0     25
                              2.0    447
                              3.0    304
                              4.0    192
                              5.0     32
...                                  ...
56         hoh_most_education 5.0      3
           racial_background  1.0    102
                              2.0     11
                              3.0      3
                              5.0     18

[458 rows x 1 columns]

Merge with sum and get percentages for each state from comscore:

In [219]:
state_vars = (state_df.merge(state_df
 .groupby(['state_fips', 'var_name'])
 .agg({'count': np.sum}), 
            how='left',
            right_index=True,
            left_on=['state_fips', 'var_name'])
 .assign(pct=lambda x: x.count_x/x.count_y)
 .rename(columns={'count_x': 'count', 'count_y': 'total'}))
state_vars.index.rename(['state', 'comscore_column', 'comscore_code'], inplace=True)

In [206]:
state_vars

count  total       pct
state_fips var_name           val                        
01         hoh_most_education 1.0     25   1000  0.025000
                              2.0    447   1000  0.447000
                              3.0    304   1000  0.304000
                              4.0    192   1000  0.192000
                              5.0     32   1000  0.032000
...                                  ...    ...       ...
56         hoh_most_education 5.0      3     86  0.034884
           racial_background  1.0    102    134  0.761194
                              2.0     11    134  0.082090
                              3.0      3    134  0.022388
                              5.0     18    134  0.134328

[458 rows x 3 columns]

In [196]:
state_vars.to_csv('../output/state_census_distributions.csv')

In [158]:
census_df.head()

,DP02_0064PE,DP02_0061PE,DP02_0065PE,DP02_0062PE,DP02_0063PE,DP02_0060PE,DP02_0059PE,DP05_0077PE,DP05_0078PE,DP05_0080PE,state
0,13.3,30.4,8.0,22.6,9.1,11.0,5.6,57.0,37.5,1.0,28
1,23.4,24.7,18.7,15.8,7.7,5.1,4.6,72.9,6.7,6.2,25
2,18.8,25.1,9.9,22.1,6.9,8.5,8.7,42.9,11.7,4.5,48
3,21.8,29.6,15.0,17.4,8.5,5.2,2.6,93.2,1.2,1.5,50
4,18.3,35.6,11.8,16.1,8.2,6.8,3.3,77.3,10.6,3.2,42


In [210]:
multi_census_df = (census_df
 .melt(id_vars='state')
 .merge(census_mapping, 
        left_on='variable', 
        right_on='census_code', 
        how='left')
 .drop('description', axis=1)
 .groupby(['state', 'comscore_column', 'census_code', 'comscore_code'])
 .agg('sum')
)

In [245]:
idx = state_vars.index
state_vars.index.set_levels(idx.levels[-1].astype(int), level=-1, inplace=True)
state_vars.index.set_levels(idx.levels[0].astype(int), level=0, inplace=True)

/tmp/ipykernel_3878524/410209035.py:2: FutureWarning: inplace is deprecated and will be removed in a future version.
  state_vars.index.set_levels(idx.levels[-1].astype(int), level=-1, inplace=True)
/tmp/ipykernel_3878524/410209035.py:3: FutureWarning: inplace is deprecated and will be removed in a future version.
  state_vars.index.set_levels(idx.levels[0].astype(int), level=0, inplace=True)


In [259]:
mdf = (state_vars.merge(multi_census_df, 
                 left_index=True, 
                 right_on=['state', 'comscore_column', 'comscore_code'])
      .assign(pct=lambda x: x.pct*100))
mdf.columns = ['comscore_count', 'comscore_total', 'comscore_pct', 'census_pct']

#(mdf.merge(census_mapping[['census_code', 'description']], left_on=['census_code'], right_on=['census_code'])


In [260]:
mdf

comscore_count  \
state comscore_column    census_code comscore_code                   
1     hoh_most_education DP02_0061PE 1                          25   
                         DP02_0062PE 2                         447   
                         DP02_0063PE 3                         304   
                         DP02_0064PE 4                         192   
                         DP02_0065PE 5                          32   
...                                                            ...   
56    hoh_most_education DP02_0064PE 4                          19   
                         DP02_0065PE 5                           3   
      racial_background  DP05_0077PE 1                         102   
                         DP05_0078PE 2                          11   
                         DP05_0080PE 3                           3   

                                                    comscore_total  \
state comscore_column    census_code comscore_code                   
1     hoh_most_education DP02_0061PE 1                        1000   
                         DP02_0062PE 2                        1000   
                         DP02_0063PE 3                        1000   
                         DP02_0064PE 4                        1000   
                         DP02_0065PE 5                        1000   
...                                                            ...   
56    hoh_most_education DP02_0064PE 4                          86   
                         DP02_0065PE 5                          86   
      racial_background  DP05_0077PE 1                         134   
                         DP05_0078PE 2                         134   
                         DP05_0080PE 3                         134   

                                                    comscore_pct  census_pct  
state comscore_column    census_code comscore_code                            
1     hoh_most_education DP02_0061PE 1                  2.500000        30.9  
                         DP02_0062PE 2                 44.700000        21.7  
                         DP02_0063PE 3                 30.400000         8.2  
                         DP02_0064PE 4                 19.200000        15.4  
                         DP02_0065PE 5                  3.200000         9.1  
...                                                          ...         ...  
56    hoh_most_education DP02_0064PE 4                 22.093023        17.4  
                         DP02_0065PE 5                  3.488372         9.3  
      racial_background  DP05_0077PE 1                 76.119403        84.3  
                         DP05_0078PE 2                  8.208955         1.0  
                         DP05_0080PE 3                  2.238806         0.8  

[403 rows x 4 columns]